# Market Prediction: XGBoost with GPU (Fit in 1min)

# Check my kernel [🔥🔥 Market Prediction: CatBoost Classifier 🔥🔥](https://www.kaggle.com/hamditarek/market-prediction-catboost-classifier) ==> Public Score: 5242.189 (version 18)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cudf
pd.set_option('display.max_columns', 500)


import time
from tqdm.notebook import tqdm

# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
import cufflinks
import cufflinks as cf
import plotly.figure_factory as ff
import os


import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time
train_cudf  = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train = train_cudf.to_pandas()
del train_cudf
features = pd.read_csv('../input/jane-street-market-prediction/features.csv')
example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
print ("Data is loaded!")

In [ ]:
print('train shape is {}'.format(train.shape))
print('features shape is {}'.format(features.shape))
print('example_test shape is {}'.format(example_test.shape))
print('sample_prediction_df shape is {}'.format(sample_prediction_df.shape))

In [ ]:
train.head()

### Missing Values Count

In [ ]:
missing_values_count = train.isnull().sum()
print (missing_values_count)
total_cells = np.product(train.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

# Is the data balanced or not?

In [ ]:
train = train[train['weight'] != 0]

train = train.query('date > 85').reset_index(drop = True) 

train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use

train['action'] = ((train['weight'].values * train['resp'].values) > 0).astype('int')

train.fillna(train.mean(),inplace=True)

cols = [c for c in train.columns if 'feature' in c]

X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = train.loc[:, 'action']

In [ ]:
x = train['action'].value_counts().index
y = train['action'].value_counts().values

trace2 = go.Bar(
     x=x ,
     y=y,
     marker=dict(
         color=y,
         colorscale = 'Viridis',
         reversescale = True
     ),
     name="Imbalance",    
 )
layout = dict(
     title="Data imbalance - action",
     #width = 900, height = 500,
     xaxis=go.layout.XAxis(
     automargin=True),
     yaxis=dict(
         showgrid=False,
         showline=False,
         showticklabels=True,
 #         domain=[0, 0.85],
     ), 
)
fig1 = go.Figure(data=[trace2], layout=layout)
iplot(fig1)

In [ ]:
del x, y, train, features, example_test, sample_prediction_df

## Training
##### To activate GPU usage, simply use tree_method='gpu_hist' (took me an hour to figure out, I wish XGBoost documentation was clearer about that).

In [ ]:
import xgboost as xgb
print("XGBoost version:", xgb.__version__)

In [ ]:
clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=11,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    missing=-999,
    random_state=2020,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
TRAINING = True

start_time = time.time()

if TRAINING:
    import janestreet
    env = janestreet.make_env()
    th = 0.5
    for (test_df, pred_df) in tqdm(env.iter_test()):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, test_df.columns.str.contains('feature')].values
            if np.isnan(x_tt[:, 1:].sum()):
                x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:])
            pred = clf.predict(x_tt)
            pred_df.action = np.where(pred >= th, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)
        
print(f"took: {time.time() - start_time} seconds")